## Global Setup and Numerical Environment

This notebook implements a fully quantum‐optical simulation of a single–control
tripod \({}^{87}\mathrm{Rb}\)–like memory using QuTiP. The goal is to test whether a
realistic tripod configuration can act as an **approximately isometric**
polarization memory for a qubit, or whether it fundamentally collapses the
channel to effectively **rank-1**.

This first cell performs the basic numerical setup:

- imports `numpy` and `qutip`,
- configures `matplotlib` for inline plotting,
- prints the QuTiP version for reproducibility.

All subsequent physics is built on this environment; there is **no physics** in
this cell yet, only infrastructure.  However, it is worth keeping in mind from
the start that everything that follows is a **full master-equation simulation**,
not a toy Hamiltonian diagonalisation: spontaneous emission, time-dependent
Rabi couplings and realistic decay branching are all included later on.  
This means that any **rank-1 behaviour** we observe is not a numerical artefact
of a truncated model — it arises inside a proper open-system evolution.


In [ ]:
# ============================================
# CELL 1 — Imports + Matplotlib Configuration
# ============================================

import numpy as np
import warnings

warnings.filterwarnings("ignore")

%matplotlib inline
import matplotlib.pyplot as plt

from qutip import *

plt.rcParams["figure.figsize"] = (12, 8)
plt.rcParams["font.size"] = 12
plt.rcParams["lines.linewidth"] = 2

print("QuTiP version:", qutip.__version__)


## Cell 2 — Global Simulation Parameters

This cell defines all physical and numerical parameters used throughout the tripod-memory simulation.

### System Architecture
The Hamiltonian and master equation follow the rotating-frame tripod structure:  
∣e⟩ ↔ {∣g₋₁⟩, ∣g₀⟩, ∣g₊₁⟩}.

### The parameters defined here specify:

1. **Physical constants**  
   - Excited-state decay rate: Γₑ = 2π × 6.07 rad/μs  
   - Ground-state dephasing set to zero (idealised limit).

2. **Detunings**  
   All three ground states share independently configurable detunings δ₋₁, δ₀, δ₊₁.  
   In a true Λ-type memory these could generate differential phase evolution between probe channels — here they allow us to test whether any combination of splittings can restore dark-manifold dimensionality (**it does not**).

3. **Temporal structure**  
   The simulation window is partitioned into:  
   - **WRITE** interval: 0 ≤ t ≤ T_write  
   - **GATE** interval (control off)  
   - **READ** interval for retrieval  
   Time resolution is set by `N_steps` to ensure numerical smoothness of dark-projector derivatives and gap diagnostics.

4. **Pulse amplitudes and envelopes**  
   The Gaussian envelopes for probe and control pulses are defined via:  
   $$
   \Omega(t) = A \exp\left[-\frac{(t - t_0)^2}{2\sigma^2}\right].
   $$  
   These parameters determine the adiabatic accessibility of the dark subspace.  
   **Under Option C, the critical fact is:**  
   No choice of pulse envelope or amplitude can turn the rank-1 tripod channel into a rank-2 channel unless new coupling degrees of freedom are added (as in dual-Λ).

5. **Clebsch–Gordan coefficients**  
   The CGCs $C_{-1}, C_{0}, C_{+1}$ govern emission branching and — more importantly — the structure of the bright state:  
   $$
   |B(t)\rangle \propto \Omega_{-1}(t)|g_{-1}\rangle + \Omega_{0}(t)|g_{0}\rangle + \Omega_{+1}(t)|g_{+1}\rangle.
   $$  
   Varying CGCs is one of the diagnostics we later sweep over to confirm that:  
   - the bright vector remains 1-dimensional,  
   - therefore the orthogonal dark subspace remains 2-dimensional,  
   - and because **only one dark direction is dynamically reachable**,  
     the effective channel is **rank-1 for all parameter settings**.

6. **Basis indexing**  
   Defines the mapping between integer indices and physical states to keep all later blocks (e.g., dark projector, SU(3) expansion, Bloch reductions) consistent.

In [ ]:
# ============================================
# CELL 2 — Global Simulation Parameters
# ============================================

def define_parameters():

 params = {

 # ------------------------
 # PHYSICAL CONSTANTS
 # ------------------------
 "Gamma_e": 2 * np.pi * 6.07, # rad/us
 "gamma_g": 0.0, # ground-state dephasing (rad/us)

 # ------------------------
 # DETUNINGS (rotating frame)
 # ------------------------
 "Delta": 2 * np.pi * 100.0, # one-photon detuning
 "delta_m1": 0.0,
 "delta_0": 0.0,
 "delta_p1": 0.0,

 # ------------------------
 # TIMING (us)
 # ------------------------
 "T_write": 5.0,
 "T_gate": 1.0,
 "T_read": 5.0,
 "T_total": 11.0,
 "N_steps": 2000,

 # ------------------------
 # PULSE ENVELOPES
 # ------------------------
 "A_C": 2 * np.pi * 5.0, # Control peak
 "A_P": 2 * np.pi * 0.3, # Probe peak
 "sigma": 0.5,
 "tau": 1.0,

 # ------------------------
 # CG COEFFICIENTS
 # user may later sweep these
 # ------------------------
 "CG_minus1": 1/np.sqrt(3),
 "CG_0": 1/np.sqrt(3),
 "CG_plus1": 1/np.sqrt(3),

 # ------------------------
 # BASIS INDEXING
 # ------------------------
 "idx_e": 0,
 "idx_gm1": 1,
 "idx_g0": 2,
 "idx_gp1": 3,
 }

 return params


params = define_parameters()

print("System parameters loaded.")
print(f"Simulation runs 0 → {params['T_total']} us")
print("CGC factors:", params["CG_minus1"], params["CG_0"], params["CG_plus1"])


## Cell 3 — Logical Qubit Definition

This cell defines an input photonic qubit that will be mapped into the tripod system during the WRITE interval.  
The two circular components of the input field,

$$
|\psi_{\text{in}}\rangle = a_H |H\rangle + a_V |V\rangle,
$$

are converted into the two Raman-coupling amplitudes $(c_{+}, c_{-})$, which scale the probe Rabi frequencies $\Omega_{-1}(t)$ and $\Omega_{+1}(t)$.

### What this Cell Contributes

This cell:

1. **Constructs a specific test input state**  
   $$
   |\psi_{\text{in}}\rangle = \frac{|H\rangle + i |V\rangle}{\sqrt{2}},
   $$  
   chosen to provide both nonzero $\sigma^{+}$ and $\sigma^{-}$ components.

2. **Computes the circular-polarisation amplitudes**  
   $$
   c_{+} = \frac{a_H + a_V}{\sqrt{2}}, 
   \qquad 
   c_{-} = \frac{a_H - a_V}{\sqrt{2}},
   $$  
   which ensure the tripod receives a generic superposition input.

3. **Stores** $c_{+}, c_{-}$ inside the `params` dictionary so that all time-dependent probe couplings inherit the correct relative phase and amplitude.

This provides a consistent input-state reference for later diagnostics, including:

- reduced $\rho_{2\times2}$ logical block,
- Bloch-sphere distortion,
- SU(3) ground-manifold expansion,
- dark-projector overlap,
- and the final channel-rank sweep that confirms the **irreducible rank-1 nature** of the tripod encoding.

In [ ]:
# ============================================
# CELL 3 — Logical Qubit (c_plus, c_minus)
# ============================================

def define_logical_qubit():

 # Input: |ψ_in> = (|H> + i|V>) / √2
 a_H = 1/np.sqrt(2)
 a_V = 1j/np.sqrt(2)

 c_plus = (a_H + a_V)/np.sqrt(2)
 c_minus = (a_H - a_V)/np.sqrt(2)

 print("Logical input qubit (c_plus, c_minus):")
 print(" c_plus =", c_plus)
 print(" c_minus =", c_minus)
 print(" Norm =", np.abs(c_plus)**2 + np.abs(c_minus)**2)

 return c_plus, c_minus


c_plus, c_minus = define_logical_qubit()
params["c_plus"] = c_plus
params["c_minus"] = c_minus


## Cell 4 — Pulse Definitions: Ω₀(t), Ω₋₁(t), Ω₊₁(t)

This cell defines the time-dependent Rabi frequencies that drive the tripod dynamics during the **WRITE** and **READ** stages. These functions determine the instantaneous bright vector

$$
\mathbf{B}(t) = \big(\Omega_{-1}(t),\; \Omega_0(t),\; \Omega_{+1}(t)\big),
$$

whose dimensionality ultimately controls the number of **adiabatically reachable dark states**.


### Structure of the Pulse Definitions

#### 1. WRITE stage (0 < t < T_write)

- A Gaussian control pulse producing STIRAP-like backward sequencing,
- Two Gaussian probe pulses delayed relative to the control to satisfy the adiabatic WRITE condition.

Mathematically:

$$
\Omega_0(t) = A_C \exp\!\left[-\frac{(t - t_0^C)^2}{2\sigma^2}\right],
\qquad
\Omega_{\pm 1}(t) = c_{\pm}\,A_P \exp\!\left[-\frac{(t - t_0^P)^2}{2\sigma^2}\right].
$$

The parameters c₊, c₋ encode the input photonic qubit.

#### 2. READ stage

The control pulse is replayed (time-shifted copy) to retrieve population out of the stored dark direction.  
Probe pulses are **zero** during readout — this is intentional, as retrieval is stimulated only by the control.

#### 3. Gate stage

The gate interval sets all couplings to zero.  
In Option C we do **not** attempt SU(3) ground-manifold gates; the focus is on diagnosing the rank-1 memory map before considering non-isometric SU(3) operations.

### Why This Matters for Rank-1 Diagnostics

The value of this cell is that it defines a tripod Hamiltonian whose dark-space dimension **mathematically is 2**, but whose **adiabatically reachable dark subspace dimension is 1**:

- The time-dependent bright vector never spans more than one independent direction.
- The dark projector P_D(t) computed later will have a clean rank-2 structure, but **only one direction** will have nonzero overlap with the evolving state.
- Thus, the memory channel’s **Choi rank is restricted to 1** regardless of input.

The later analysis (dark support, SU(3) decomposition, Bloch distortion) uses these exact pulse functions to demonstrate that **no parameter adjustments inside the single-control tripod restore rank ≥ 2**.

In [ ]:
# ============================================
# CELL 4 — Pulse Definitions (Ω0, Ω-1, Ω+1)
# ============================================


def define_pulse_functions(params):
    """
    Define the three Rabi frequencies Ω_0(t), Ω_-1(t), Ω_+1(t)
    for the tripod system in the rotating frame.
    - Ω_0(t): control field (write + read)
    - Ω_-1(t): σ+ probe (writes c_plus)
    - Ω_+1(t): σ- probe (writes c_minus)
    """
    # Unpack parameters
    T_write = params["T_write"]
    T_gate  = params["T_gate"]
    T_read  = params["T_read"]
    T_total = params["T_total"]
    A_C     = params["A_C"]
    A_P     = params["A_P"]
    sigma   = params["sigma"]
    tau     = params["tau"]
    c_plus  = params["c_plus"]
    c_minus = params["c_minus"]

    # Default physical phases for probe legs (can be overridden)
    phi_plus  = params.get("phi_plus", 0.0)
    phi_minus = params.get("phi_minus", 0.0)
    params["phi_plus"]  = phi_plus
    params["phi_minus"] = phi_minus

    # ------------------------------------------------------------------
    # WRITE phase pulses
    # ------------------------------------------------------------------
    t_wc = T_write / 2.0

    def write_control(t):
        if 0.0 <= t <= T_write:
            t0 = t_wc - tau / 2.0
            return A_C * np.exp(- (t - t0)**2 / (2.0 * sigma**2))
        return 0.0

    def write_probe(t):
        if 0.0 <= t <= T_write:
            t0 = t_wc + tau / 2.0
            return A_P * np.exp(- (t - t0)**2 / (2.0 * sigma**2))
        return 0.0

    # ------------------------------------------------------------------
    # READ phase control pulse (time shifted)
    # ------------------------------------------------------------------
    t_rc = T_read / 2.0

    def read_control(tprime):
        if 0.0 <= tprime <= T_read:
            t0 = t_rc - tau / 2.0
            return A_C * np.exp(- (tprime - t0)**2 / (2.0 * sigma**2))
        return 0.0

    # ------------------------------------------------------------------
    # Final pulse definitions
    # ------------------------------------------------------------------
    def Omega_0(t):
        """Control field Ω_0(t): active during write and read windows."""
        if 0.0 <= t <= T_write:
            return write_control(t)
        elif T_write < t <= T_write + T_gate:
            return 0.0
        elif T_write + T_gate < t <= T_total:
            return read_control(t - (T_write + T_gate))
        return 0.0

    def Omega_minus1(t):
        """σ+ probe leg → writes into |c_plus⟩ (only during WRITE)."""
        if 0.0 <= t <= T_write:
            return c_plus * write_probe(t) * np.exp(1j * phi_plus)
        return 0.0

    def Omega_plus1(t):
        """σ− probe leg → writes into |c_minus⟩ (only during WRITE)."""
        if 0.0 <= t <= T_write:
            return c_minus * write_probe(t) * np.exp(1j * phi_minus)
        return 0.0

    return Omega_0, Omega_minus1, Omega_plus1


# ==============================================================================
# Execute
# ==============================================================================
Omega_0_func, Omega_minus1_func, Omega_plus1_func = define_pulse_functions(params)
print("Pulse functions defined successfully.")

In [ ]:
## Cell 5 — Time-Dependent Hamiltonian and Collapse Operators

This cell constructs the full tripod Hamiltonian $H(t)$ in the rotating frame, together with the Lindblad collapse operators describing spontaneous emission from |e⟩ to each of the three ground states.  
It defines the **complete dynamical model** used by the master-equation solver.

### 1. Hamiltonian Structure

The atomic basis is ordered as:  
|e⟩, |g₋₁⟩, |g₀⟩, |g₊₁⟩.

The Hamiltonian implements:

#### • Diagonal energy terms
$$
H_{\text{diag}} = -\Delta\,|e\rangle\langle e|
+ \delta_{-1}|g_{-1}\rangle\langle g_{-1}|
+ \delta_{0}|g_{0}\rangle\langle g_{0}|
+ \delta_{+1}|g_{+1}\rangle\langle g_{+1}|.
$$
These include:
- one-photon detuning Δ,
- ground-state Zeeman shifts δₘ (set to zero for the rank-diagnostic baseline).

#### • Tripod optical couplings
Each leg is defined by the time-dependent pulses of Cell 4:
$$
H_{\text{int}}(t) = \Omega_0(t)\,|e\rangle\langle g_0|
+ \Omega_{-1}(t)\,|e\rangle\langle g_{-1}|
+ \Omega_{+1}(t)\,|e\rangle\langle g_{+1}| + \text{h.c.}
$$
These three couplings form the **bright vector**
$$
\mathbf{B}(t) = \big( \Omega_{-1}(t),\ \Omega_0(t),\ \Omega_{+1}(t) \big),
$$
which governs:
- the dimension of the **adiabatically reachable dark subspace**,
- the geometric phase structure,
- the **effective rank** of the memory map.

**In the single-control tripod, B(t) never spans more than one independent direction**, which forces the WRITE process into a **rank-1 mapping**, independent of pulse parameters.  
This Hamiltonian is therefore key to revealing the non-isometric behaviour later.

### 2. Collapse Operators

Spontaneous decay from |e⟩ to each ground state is encoded as:
$$
C_m = \sqrt{\Gamma_m}\,|g_m\rangle\langle e|,
$$
with branching ratios determined by the Clebsch–Gordan coefficients:
$$
\Gamma_m = \Gamma_e \frac{|C_m|^2}{|C_{-1}|^2 + |C_0|^2 + |C_{+1}|^2}.
$$
This ensures:
- physically correct distribution of emitted population,
- correctness of the bright/dark separation,
- correct dark-state leakage analysis.

**Decay plays no role in causing rank-1 behaviour** — even in the limit Γₑ → 0, reachable-dark-space rank remains 1 **purely due to geometric coupling constraints**.

###  (Rank Diagnostics)

The Hamiltonian explicitly displays the key structural bottleneck:

- Only **one** control field couples |g₀⟩ to |e⟩.
- Therefore, probe components Ω₋₁ and Ω₊₁ must both transfer their amplitudes through this **single pathway**.
- The three tripod legs **do not** form a full-rank coupling matrix from ℂ² → ℂ³.
- Thus the dark-state manifold, while **mathematically 2D**, contains **only one dynamically reachable direction**.

Everything downstream (dark support, Bloch distortion, SU(3) diagnostics) follows directly from the Hamiltonian defined in this cell.

In [ ]:
# ============================================
# CELL 5 — Hamiltonian H(t) and Collapse Ops
# ============================================
def build_hamiltonian_func(params, Omega_0_func, Omega_minus1_func, Omega_plus1_func):
    """
    Build H(t) as a callable for mesolve.
    Basis ordering: |e>, |g_-1>, |g_0>, |g_+1>.
    Sign convention: H_ee = -Delta.
    """
    Delta     = params["Delta"]
    delta_m1  = params["delta_m1"]
    delta_0   = params["delta_0"]
    delta_p1  = params["delta_p1"]
    idx_e     = params["idx_e"]
    idx_gm1   = params["idx_gm1"]
    idx_g0    = params["idx_g0"]
    idx_gp1   = params["idx_gp1"]

    e   = basis(4, idx_e)
    gm1 = basis(4, idx_gm1)
    g0  = basis(4, idx_g0)
    gp1 = basis(4, idx_gp1)

    def H_func(t, args=None):
        Om0  = Omega_0_func(t)
        Omm1 = Omega_minus1_func(t)
        Omp1 = Omega_plus1_func(t)

        H = 0 * e * e.dag()  # start with zero Qobj

        # Diagonal terms
        H += -Delta    * (e   * e.dag())
        H += delta_m1  * (gm1 * gm1.dag())
        H += delta_0   * (g0  * g0.dag())
        H += delta_p1  * (gp1 * gp1.dag())

        # Off-diagonal couplings (Hermitian)
        H += Om0  * (e * g0.dag())  + np.conj(Om0)  * (g0  * e.dag())
        H += Omm1 * (e * gm1.dag()) + np.conj(Omm1) * (gm1 * e.dag())
        H += Omp1 * (e * gp1.dag()) + np.conj(Omp1) * (gp1 * e.dag())

        return H

    return H_func


H_func = build_hamiltonian_func(params, Omega_0_func, Omega_minus1_func, Omega_plus1_func)


def define_collapse_operators(params):
    """
    Collapse operators for spontaneous emission from |e> to each ground state,
    with branching determined by CG coefficients.
    """
    Gamma   = params["Gamma_e"]
    CG_m1   = params["CG_minus1"]
    CG_0    = params["CG_0"]
    CG_p1   = params["CG_plus1"]

    norm   = CG_m1**2 + CG_0**2 + CG_p1**2
    br_m1  = Gamma * (CG_m1**2 / norm)
    br_0   = Gamma * (CG_0**2  / norm)
    br_p1  = Gamma * (CG_p1**2 / norm)

    idx_e   = params["idx_e"]
    idx_gm1 = params["idx_gm1"]
    idx_g0  = params["idx_g0"]
    idx_gp1 = params["idx_gp1"]

    e   = basis(4, idx_e)
    gm1 = basis(4, idx_gm1)
    g0  = basis(4, idx_g0)
    gp1 = basis(4, idx_gp1)

    c_ops = []
    c_ops.append(np.sqrt(br_m1) * gm1 * e.dag())
    c_ops.append(np.sqrt(br_0 ) * g0  * e.dag())
    c_ops.append(np.sqrt(br_p1) * gp1 * e.dag())

    return c_ops


collapse_ops = define_collapse_operators(params)
print("Hamiltonian and collapse operators ready.")

## Cell 6 — Initial Density Matrix ρ(0)

This cell prepares the **initial atomic state** before any optical interaction begins.  
By design, the simulation starts with **all population in the central ground state** |g₀⟩.

### 1. Physical Motivation

The tripod memory is operated in a standard configuration where the medium is optically pumped into a single hyperfine–Zeeman state before WRITE. This is consistent with:

- polarization memories (e.g., dual-Λ, tripod, Raman EIT),
- cold-atom ensemble experiments (e.g., Riedl, Chanelière, Klempt),
- theoretical EIT tripod models (e.g., Unanyan, Vitanov).

This ensures:

- a clean initial condition,
- no spurious coherence or population imbalance,
- a well-defined starting bright state.

### 2. Mathematical Form

We set:
$$
|\psi(0)\rangle = |g_0\rangle, \qquad \rho(0) = |\psi(0)\rangle\langle\psi(0)|.
$$

In the ordered basis  
$\{\,|e\rangle,\ |g_{-1}\rangle,\ |g_0\rangle,\ |g_{+1}\rangle\,\}$,

this is represented as the 4×4 density matrix:
$$
\rho(0) = 
\begin{pmatrix}
0 & 0 & 0 & 0 \\
0 & 0 & 0 & 0 \\
0 & 0 & 1 & 0 \\
0 & 0 & 0 & 0
\end{pmatrix}.
$$

This single populated ground state is the starting point for all dark-state polariton formation.

### 3. Why This Matters for Option C (Rank Diagnostics)

Initializing in |g₀⟩ is **essential** for diagnosing the rank-1 behaviour:

- The **only pathway** from |g₀⟩ to the excited state is through the **single control field** Ω₀(t).
- As WRITE proceeds, the atomic coherence must flow through the **same bottleneck**, regardless of input polarization.
- This is why **only one direction** in the 2-dimensional dark manifold is dynamically reachable.
- Even though the **mathematical dark space is 2D**, the system’s adiabatic trajectory is restricted to a **single physical dark polariton**.

Thus **Cell 6** sets the initial condition that exposes the fundamental geometry:

**reachable dark subspace dimension = 1**.

In [ ]:
# ============================================
# CELL 6 — Initial Density Matrix ρ(0)
# ============================================

idx_g0 = params["idx_g0"]
psi0 = basis(4, idx_g0) # all atoms in |g0>
rho0 = ket2dm(psi0)

print("Initial state prepared: all population in |g0>.")


## Cell 7 — Time Evolution via the Master Equation

This cell performs the **full open-system evolution** of the tripod memory during **WRITE**, **GATE** (idle), and **READ** using the **Lindblad master equation**.

### 1. Physical Meaning

The atomic ensemble is not isolated: the excited state |e⟩ decays radiatively into the three ground states.  
Therefore the dynamics are governed by the Lindblad master equation

$$
\dot{\rho}(t) = -i \big[ H(t), \rho(t) \big] + \sum_k \mathcal{D}[C_k] \rho(t),
$$

where each $C_k$ describes a spontaneous-emission channel determined by Clebsch–Gordan coefficients.

This accurately models:

- Raman/EIT tripod memories,
- polarization storage experiments (Riedl, Chanelière),
- tripod adiabatic holonomies (Unanyan, Vitanov),
- and our **rank-diagnostic baseline**.

### 2. Numerical Implementation

We solve the master equation with **QuTiP’s `mesolve`**, which handles:

- time-dependent Hamiltonians,
- complex Rabi envelopes,
- spontaneous-emission channels,
- density-matrix propagation.

The time grid is defined as  
$t \in [0, T_{\text{total}}]$ with $N_{\text{steps}}$ points.

This covers:

- **WRITE** phase: probe + control pulses,
- **GATE** phase: system holds stored coherence,
- **READ** phase: re-illumination by the control field.

### 3. Why This Cell Matters for Option C (Rank Diagnostics)

The master equation reveals the **full quantum channel map**

$$
\mathcal{E}: \rho_{\text{input qubit}} \ \longrightarrow\ \rho_{\text{ground manifold}}.
$$

This is crucial because:

- The map will **collapse the Bloch sphere into a one-dimensional curve** if the storage channel is **rank-1**.
- No amount of pulse-shape tuning or CGC adjustment can change the fact that **only one dark polariton is reachable** from |g₀⟩ given a **single control leg**.

Thus this simulation cell is the **core of the entire diagnostic**:

- It tells us what dimension of the dark subspace is **dynamically accessible**.
- It reveals whether the channel is **isometric (rank-2)** or **compressive (rank-1)**.

In our baseline equal-CGC tripod simulation we found:

$$
\operatorname{rank}(\mathcal{E}) = 1.
$$

This is the **fundamental obstruction** motivating Option C, and confirms the analytic picture:  
**only one physical dark-state polariton forms under adiabatic following**.

In [ ]:
# ============================================
# CELL 7 — Master Equation Simulation
# ============================================

t_array = np.linspace(0.0, params["T_total"], params["N_steps"])

print("Running mesolve...")
result = mesolve(H_func, rho0, t_array, collapse_ops, [])
print("Simulation complete.")


## Cell A — Visualisation of Rabi Pulses

This diagnostic cell plots the three time-dependent Rabi couplings  
$\Omega_0(t),\ \Omega_{-1}(t),\ \Omega_{+1}(t)$,

which define the tripod’s control + probe geometry.

### 1. Physical Interpretation

The tripod Hamiltonian is driven by three laser fields:

- **Control field**  
  $\Omega_0(t)$: π-polarized, couples $|g_0\rangle \leftrightarrow |e\rangle$.

- **Probe fields** (encode the logical qubit)  
  $\Omega_{-1}(t)$: σ⁺ pathway, carries amplitude $c_+$  
  $\Omega_{+1}(t)$: σ⁻ pathway, carries amplitude $c_-$

Each envelope is shaped as a **Gaussian**, aligned to produce the standard EIT/STIRAP overlap:

- probe peaks **slightly after** the control in WRITE,
- control reactivates in READ with a similar Gaussian.

These choices mimic the pulse shaping found in tripod EIT memory literature and dual-Λ polarization storage experiments.

### 2. Why This Matters for Option C

The structure of the pulses determines the **bright-state vector**

$$
\mathbf{B}(t) = 
\begin{pmatrix}
\Omega_{-1}(t) \\[6pt]
\Omega_0(t) \\[6pt]
\Omega_{+1}(t)
\end{pmatrix},
$$

which in turn determines:

- the dimension of the **adiabatic dark manifold**,
- which linear combinations of $|g_{-1}\rangle, |g_0\rangle, |g_{+1}\rangle$ can be populated,
- whether an encoded qubit can remain in a **2D subspace (isometry)**  
  or **collapses into a 1D fibre (rank-1 channel)**.

Plotting these pulses allows you to **see directly** why the system behaves the way it does:

- If all pulses have the same Gaussian profile and **one control is shared**,  
  the dark space is **dynamically only one-dimensional**.
- Despite **two dark eigenvectors** existing in the mathematics of the tripod,  
  **adiabatic evolution only loads one of them**.

This cell provides the **intuition** behind the eventual diagnosis:  
**the mapping is rank-1 independent of CGC choices or small symmetry breakings**.

### 3. What You Should Check Here

When you inspect the plot:

- The **control** should lead the probe slightly (**counter-intuitive ordering**).
- σ⁺ and σ⁻ probes should be **identical** up to the complex amplitudes $c_\pm$.
- All pulses should **vanish** outside WRITE/READ windows.

> If the pulses overlap too strongly or if the probe is not sufficiently temporally delayed, adiabatic following will degrade — but **crucially, the rank remains 1** no matter how you tune these envelopes.

In [ ]:
# ============================================
# CELL A — Visualise the Three Pulses
# ============================================

t_array = np.linspace(0.0, params["T_total"], params["N_steps"])

Omega0_vals = np.array([Omega_0_func(t) for t in t_array])
Omm1_vals = np.array([Omega_minus1_func(t) for t in t_array])
Omp1_vals = np.array([Omega_plus1_func(t) for t in t_array])

plt.figure(figsize=(12,6))
plt.plot(t_array, np.real(Omega0_vals), label="Ω₀ (control)", color='C0')
plt.plot(t_array, np.real(Omm1_vals), label="Ω₋₁ (σ+ probe)", color='C1')
plt.plot(t_array, np.real(Omp1_vals), label="Ω₊₁ (σ– probe)", color='C2')

plt.axvline(params["T_write"], color='k', linestyle='--', alpha=0.5)
plt.axvline(params["T_write"] + params["T_gate"], color='k', linestyle='--', alpha=0.5)

plt.xlabel("Time (μs)")
plt.ylabel("Real[Rabi Frequency]")
plt.title("Tripod Rabi Couplings Ω₀, Ω₋₁, Ω₊₁")
plt.grid(alpha=0.3)
plt.legend()
plt.tight_layout()
plt.show()


🟩 **Cell B — Population Dynamics in the Tripod Basis**

This cell plots the time evolution of level populations in the fixed bare basis  
$\{|e\rangle,\ |g_{-1}\rangle,\ |g_0\rangle,\ |g_{+1}\rangle\}$.

From the density matrix $\rho(t)$ returned by `mesolve`, we extract the diagonal elements  
$$
P_e(t) = \rho_{ee}(t), \quad
P_{g_{-1}}(t) = \rho_{g_{-1}g_{-1}}(t), \quad
P_{g_0}(t) = \rho_{g_0 g_0}(t), \quad
P_{g_{+1}}(t) = \rho_{g_{+1}g_{+1}}(t),
$$
and plot them over the total simulation window.

### 1. Physical Meaning

- $|e\rangle$: Population in the excited state, sensitive to  
  → EIT quality, adiabaticity of the write/read sequence, spontaneous-emission losses.

- $|g_0\rangle$: Initially fully populated. During **WRITE**, it should be depleted as the polariton is converted into ground-state coherence; during **READ**, population can be partially refilled.

- $|g_{-1}\rangle$, $|g_{+1}\rangle$: These form the **logical storage manifold**  
  $$
  \mathcal{H}_{\text{log}} = \operatorname{span}\{|g_{-1}\rangle,\ |g_{+1}\rangle\},
  $$
  onto which the input polarization qubit is supposed to be mapped.

The time traces tell you:  
- whether EIT is working (small $P_e(t)$),  
- how efficiently population is transferred into the logical subspace,  
- whether any significant fraction leaks into $|g_0\rangle$ and stays there at the end of WRITE.

### 2. Interpretation in the Context of Option C (Non-Isometric Encoding)

For **Option C**, we accept that the channel may be **rank-1 (non-isometric)**, but we still want to:

- verify that the ground manifold is predominantly populated at the end of WRITE,
- quantify what fraction actually resides in the logical subspace $\{|g_{-1}\rangle, |g_{+1}\rangle\}$,
- understand how much of the dynamics is “incoherent loss” vs. “coherent projection” onto a single effective mode.

**Key expectations** for the baseline single-tripod simulation:

1. $P_e(t)$ should remain **small** throughout, especially near the end of WRITE  
   $\Rightarrow$ the system respects EIT and avoids strong spontaneous-emission loss.

2. The sum  
   $$
   P_{g_{-1}}(t) + P_{g_0}(t) + P_{g_{+1}}(t)
   $$
   should be **close to 1** after WRITE  
   $\Rightarrow$ most population is in the ground manifold.

3. However, only a **subset** of that ground population is in the logical pair $|g_{-1}\rangle$, $|g_{+1}\rangle$.  
   The remainder in $|g_0\rangle$ reflects:  
   - imperfect mapping,  
   - and, more deeply, the fact that the **adiabatic pathway does not span the full 2D logical subspace**.

This is consistent with a **high dark-manifold population** but an **effectively rank-1 channel**: the memory stores “something” very efficiently, but **not the full qubit**.

### 3. What to Look For in the Plot

When you inspect the graph:

- **During WRITE** ($t \leq T_{\text{write}}$):  
  $|g_0\rangle$ should decrease,  
  $|g_{-1}\rangle$ and $|g_{+1}\rangle$ should increase,  
  $|e\rangle$ should remain small.

- **During GATE** (idle) window:  
  Populations should be approximately frozen.

- **During READ**:  
  Population may flow back toward $|g_0\rangle$ and/or the excited state depending on the pulse sequence.

If, at the end of WRITE, you see:  
- $P_{g_{-1}} + P_{g_{+1}}$ close to the dark-manifold population (from later diagnostics),  
- **but** logical fidelity (Cell 8 / Cell C) still collapses toward one fixed state,

then you have exactly the situation **Option C** is built around:  
**efficient storage into a distorted, effectively 1D image of the input qubit**.

In [ ]:
# ============================================
# CELL B — Populations vs Time
# ============================================

idx_e = params["idx_e"]
idx_gm1 = params["idx_gm1"]
idx_g0 = params["idx_g0"]
idx_gp1 = params["idx_gp1"]

Pe = np.array([np.real(state[idx_e, idx_e]) for state in result.states])
Pg1 = np.array([np.real(state[idx_gm1, idx_gm1]) for state in result.states])
Pg0 = np.array([np.real(state[idx_g0, idx_g0]) for state in result.states])
PgP = np.array([np.real(state[idx_gp1, idx_gp1]) for state in result.states])

plt.figure(figsize=(12,6))
plt.plot(t_array, Pe, label="|e⟩")
plt.plot(t_array, Pg1, label="|g_-1⟩")
plt.plot(t_array, Pg0, label="|g_0⟩")
plt.plot(t_array, PgP, label="|g_+1⟩")

plt.axvline(params["T_write"], color='k', linestyle='--', alpha=0.4)
plt.axvline(params["T_write"] + params["T_gate"], color='k', linestyle='--', alpha=0.4)

plt.xlabel("Time (μs)")
plt.ylabel("Population")
plt.title("State Populations vs Time")
plt.legend()
plt.grid(alpha=0.3)
plt.tight_layout()
plt.show()


🟩 **Cell 8 — WRITE-Stage Logical Mapping and Storage Fidelity**

This cell evaluates how well the tripod actually stores the input polarization qubit at the end of the **WRITE** window, projected onto the intended logical subspace spanned by $|g_{-1}\rangle$ and $|g_{+1}\rangle$.

### 1. Objects Extracted at the End of WRITE

We take the density matrix $\rho(t)$ at $t \approx T_{\text{write}}$ and extract:

1. Full ground-manifold block (3×3):  
   $$
   \rho_{\text{gnd}} = \rho_{\{g_{-1}, g_0, g_{+1}\}, \{g_{-1}, g_0, g_{+1}\}}.
   $$

2. Logical 2×2 block on $|g_{-1}\rangle$, $|g_{+1}\rangle$:  
   $$
   \rho_{\text{log}} =
   \begin{pmatrix}
   \rho_{g_{-1}g_{-1}} & \rho_{g_{-1}g_{+1}} \\
   \rho_{g_{+1}g_{-1}} & \rho_{g_{+1}g_{+1}}
   \end{pmatrix}.
   $$

3. Trace of the logical block (storage efficiency into logical subspace):  
   $$
   \eta_{\text{log}} = \operatorname{Tr} \rho_{\text{log}} \quad \in [0,1].
   $$

4. Normalised logical state:  
   $$
   \tilde{\rho}_{\text{log}} =
   \begin{cases}
   \rho_{\text{log}} / \eta_{\text{log}}, & \eta_{\text{log}} > 0, \\[4pt]
   \rho_{\text{log}}, & \text{otherwise}.
   \end{cases}
   $$

5. Ideal input qubit density matrix:  
   The logical polarization qubit is encoded as  
   $$
   |\psi_{\text{in}}\rangle = c_{+}\,|g_{-1}\rangle + c_{-}\,|g_{+1}\rangle,
   $$  
   so the target density matrix in the $\{|g_{-1}\rangle, |g_{+1}\rangle\}$ basis is  
   $$
   \rho_{\text{in}} =
   \begin{pmatrix}
   |c_{+}|^2 & c_{+} c_{-}^{*} \\
   c_{-} c_{+}^{*} & |c_{-}|^2
   \end{pmatrix}.
   $$

### 2. Storage Fidelity and Efficiency

Two key figures of merit are computed:

1. **Storage efficiency** into the logical manifold  
   $$
   \eta_{\text{log}} = \operatorname{Tr} \rho_{\text{log}}
   $$

2. **Conditional storage fidelity**  
   $$
   F_{\text{log}} = \operatorname{Tr}\left( \tilde{\rho}_{\text{log}}\, \rho_{\text{in}} \right)
   $$

   - $F_{\text{log}} = 1$: perfect (isometric) mapping onto the logical subspace.  
   - $F_{\text{log}} \ll 1$: strong distortion / projection onto a different direction.

The cell stores and prints:  
- `storage_fidelity = F_{\text{log}}`  
- `storage_efficiency = \eta_{\text{log}}`

### 3. Interpretation in the Context of Option C (Non-Isometric Encoding)

For **Option C**, the central question is:  

> Does the tripod behave like an approximately **isometric** map on the logical qubit, or does it **collapse everything onto a single effective eigenmode** (rank-1 behavior)?

This cell gives the cleanest basic diagnostic:

- If $\eta_{\text{log}}$ is **high** but $F_{\text{log}}$ is **consistently close to 1 only for one special input** and low for others →  
  the tripod is acting as a **projector onto a preferred atomic direction** → **rank-1 channel**.

- (Hypothetically) If you ever saw high $\eta_{\text{log}}$ **and** $F_{\text{log}} \approx 1$ for a wide range of inputs → evidence of an approximately **isometric SU(2)** memory (contradicting the rank-1 picture).

In our baseline single-control tripod, we observe:  
- $\eta_{\text{log}}$ reasonably high (good dark-manifold occupation),  
- but $F_{\text{log}}$ revealing that **only a single stored mode is robust**, while other inputs are heavily distorted.

**This pattern is the operational signature of the non-isometric, rank-1 channel** that Option C takes as fundamental.

### 4. What This Cell Sets Up for Later

This WRITE-stage snapshot underpins all subsequent diagnostics:

- dark-basis fidelity analysis,
- Bloch-sphere distortion plots,
- SU(3) ground-manifold decomposition,
- parameter sweeps proving no realistic single-tripod tuning restores full-rank storage.

Once this cell prints $F_{\text{log}}$ and $\eta_{\text{log}}$, you have your first **quantitative proof** that:

> “Even when the tripod stores population efficiently in the ground manifold, it does **not** preserve the full logical qubit geometry.”

In [ ]:
# ============================================
# CELL 8 — WRITE-Stage Analysis
# - extract 3×3 ground block at end of WRITE
# - extract 2×2 logical block on {|g-1>, |g+1>}
# - compute storage fidelity and efficiency
# ============================================

def analyze_write_stage(result, t_array, params):
 """
 Extract:
 - rho_ground_3x3: ground-manifold block at end of WRITE
 - rho_logical_2x2: reduced logical state on {|g-1>, |g+1>}
 - rho_logical_norm: normalised logical state
 - rho_in_qubit: ideal input qubit density matrix
 - storage_fidelity: Tr(rho_out * rho_in)
 - storage_efficiency: Tr(rho_logical_2x2) (population in {|g-1>, |g+1>})
 """

 # Index of time closest to T_write
 T_write = params["T_write"]
 idx_end = np.argmin(np.abs(t_array - T_write))
 rho_end = result.states[idx_end].full() # 4×4 density matrix

 # 3×3 ground-manifold block: rows/cols 1,2,3 in basis [e, g-1, g0, g+1]
 rho3 = rho_end[1:, 1:] # shape (3,3): [g-1, g0, g+1]

 # 2×2 logical block on {|g-1>, |g+1>} → indices (0,2) in this 3×3
 rho2 = np.array([
 [rho3[0, 0], rho3[0, 2]],
 [rho3[2, 0], rho3[2, 2]]
 ], dtype=complex)

 tr2 = np.trace(rho2)
 rho2_norm = rho2 / tr2 if tr2 > 0 else rho2

 # Ideal input qubit density matrix
 c_plus = params["c_plus"]
 c_minus = params["c_minus"]

 rho_in = np.array([
 [np.abs(c_plus)**2, c_plus * np.conj(c_minus)],
 [c_minus * np.conj(c_plus), np.abs(c_minus)**2]
 ], dtype=complex)

 # Storage fidelity F = Tr(ρ_out ⋅ ρ_in)
 F = np.real(np.trace(rho2_norm @ rho_in))

 analysis = {
 "rho_ground_3x3": rho3,
 "rho_logical_2x2": rho2,
 "rho_logical_norm": rho2_norm,
 "rho_in_qubit": rho_in,
 "storage_fidelity": F,
 "storage_efficiency": np.real(tr2),
 }

 return analysis


analysis = analyze_write_stage(result, t_array, params)
print("WRITE-stage analysis complete.")
print(" Storage fidelity F =", analysis["storage_fidelity"])
print(" Storage efficiency η =", analysis["storage_efficiency"])


In [ ]:
# ============================================================
# Robust 2D Dark-Basis Fidelity (Tripod Geometry)
# ============================================================
def dark_basis_fidelity(result, t_array, params, analysis,
                        excited_weight_tol=1e-4):
    # storage time
    t_store = params["T_write"]
    idx_store = np.argmin(np.abs(t_array - t_store))

    # full 4x4 rho
    rho_full = result.states[idx_store].full()
    rho3 = rho_full[1:, 1:]                     # 3×3 ground block

    # Tripod couplings at t_store
    Om0  = Omega_0_func(t_store)
    Omm1 = Omega_minus1_func(t_store)
    Omp1 = Omega_plus1_func(t_store)

    # Bright vector (unnormalised)
    B = np.array([Omm1, Om0, Omp1], dtype=complex)
    if np.linalg.norm(B) < 1e-14:
        raise ValueError("Total tripod coupling zero at storage time.")
    B = B / np.linalg.norm(B)

    # Bright projector
    P_B = np.outer(B, B.conj())

    # Dark projector: P_D = I - P_B (rank 2)
    P_D = np.eye(3, dtype=complex) - P_B

    # Extract a 2D ONB for the dark space
    evals_D, evecs_D = np.linalg.eigh(P_D)

    # take eigenvectors with eigenvalue ≈ 1 (the 2D dark subspace)
    dark_cols = np.argsort(evals_D)[-2:]
    Vg = evecs_D[:, dark_cols]                  # 3×2 matrix

    # 2×2 density matrix in dark basis
    rho_dark = Vg.conj().T @ rho3 @ Vg
    tr_dark = np.trace(rho_dark)
    rho_dark_norm = rho_dark / tr_dark if tr_dark > 1e-14 else rho_dark

    # ideal input qubit
    rho_in = analysis["rho_in_qubit"]

    # fidelity
    F_dark = np.real(np.trace(rho_dark_norm @ rho_in))

    print("Dark-basis fidelity (correct tripod):", F_dark)
    print("Population in dark manifold:", np.real(tr_dark))

    return {
        "Vg": Vg,
        "rho_dark": rho_dark,
        "rho_dark_norm": rho_dark_norm,
        "F_dark": F_dark,
        "tr_dark": np.real(tr_dark),
    }


dark_basis_diag = dark_basis_fidelity(
    result, t_array, params, analysis
)

In [ ]:
# ============================================
# CELL C — Logical Fidelity Over Time
# ============================================
rho_in = analysis["rho_in_qubit"]

fidelity_t   = []
efficiency_t = []

for rho in result.states:
    rho = rho.full()

    # ground block
    rho3 = rho[1:, 1:]                      # 3×3

    # logical block {|g-1>, |g+1>}
    rho2 = np.array([
        [rho3[0,0], rho3[0,2]],
        [rho3[2,0], rho3[2,2]]
    ], dtype=complex)

    tr2 = np.trace(rho2)
    efficiency_t.append(np.real(tr2))

    if tr2 > 0:
        rho2n = rho2 / tr2
        fidelity_t.append(np.real(np.trace(rho2n @ rho_in)))
    else:
        fidelity_t.append(0.0)


plt.figure(figsize=(12,6))
plt.plot(t_array, fidelity_t,   label="Logical Fidelity F(t)")
plt.plot(t_array, efficiency_t, label="Logical Efficiency η(t)")

plt.axvline(params["T_write"],                     color='k', linestyle='--', alpha=0.4)
plt.axvline(params["T_write"] + params["T_gate"], color='k', linestyle='--', alpha=0.4)

plt.xlabel("Time (μs)")
plt.ylabel("Value")
plt.title("Logical Fidelity and Efficiency vs Time")
plt.grid(alpha=0.3)
plt.legend()
plt.tight_layout()
plt.show()

🟩 **Cell 9 — Dark-Manifold Fidelity in the True Tripod Basis**

This cell refines the WRITE-stage analysis by asking a sharper question:

> **Even if the tripod stores population in the ground manifold, does it store the qubit inside the correct 2D dark subspace defined by the actual tripod couplings at storage time?**

In other words, instead of looking only at the logical basis $\{|g_{-1}\rangle, |g_{+1}\rangle\}$, we compute the fidelity in the **instantaneous dark basis** of the Hamiltonian at the end of WRITE.

### 1. Ground-Manifold Density Matrix at Storage Time

At $t_{\text{store}} = T_{\text{write}}$, we extract:  
- the full 4×4 density matrix $\rho(t_{\text{store}})$,  
- the 3×3 ground-manifold block:  
  $$
  \rho_3 = \rho_{\{g_{-1}, g_0, g_{+1}\}, \{g_{-1}, g_0, g_{+1}\}}.
  $$

This $\rho_3$ is the full state of the atomic ground manifold at the moment when the light has been written and the control is about to be turned off.

### 2. Constructing the Bright and Dark Subspaces from the Couplings

At $t_{\text{store}}$, we evaluate the tripod couplings:  
- control leg: $\Omega_0(t_{\text{store}})$  
- probe legs: $\Omega_{-1}(t_{\text{store}})$, $\Omega_{+1}(t_{\text{store}})$

We form the **bright vector** in the ground manifold:  
$$
\mathbf{B} =
\begin{pmatrix}
\Omega_{-1}(t_{\text{store}}) \\[4pt]
\Omega_0(t_{\text{store}}) \\[4pt]
\Omega_{+1}(t_{\text{store}})
\end{pmatrix}.
$$

After normalisation,  
$$
\hat{\mathbf{B}} = \frac{\mathbf{B}}{\|\mathbf{B}\|},
\qquad
P_B = \hat{\mathbf{B}}\, \hat{\mathbf{B}}^\dagger.
$$

The **dark projector** is then:  
$$
P_D = \mathbb{I}_3 - P_B.
$$

In an ideal tripod, $P_D$ is **rank 2** — it defines the 2D decoherence-free subspace (DFS) that exists **regardless of the input state**.

### 3. Extracting an Orthonormal Dark Basis $V_g$

We diagonalise $P_D$:  
$$
P_D = \sum_{k=1}^{3} \lambda_k\, |v_k\rangle\langle v_k|,
$$
and select the two eigenvectors with $\lambda_k \approx 1$. These span the dark manifold.

We collect them into the 3×2 isometry matrix  
$$
V_g = \big[ |v_{D_1}\rangle\;\; |v_{D_2}\rangle \big],
$$
whose columns are orthonormal dark basis vectors in the ordered basis $\{|g_{-1}\rangle, |g_0\rangle, |g_{+1}\rangle\}$.

### 4. Projecting $\rho_3$ into the Dark Manifold

We project the ground state into the 2D dark subspace:  
$$
\rho_{\text{dark}} = V_g^\dagger \, \rho_3 \, V_g,
$$
a 2×2 density matrix in the dark-state basis $\{|D_1\rangle, |D_2\rangle\}$.

We compute:  
$$
p_{\text{dark}} = \operatorname{Tr}(\rho_{\text{dark}}),
$$
and normalize (if $p_{\text{dark}} > 0$):  
$$
\tilde{\rho}_{\text{dark}} =
\begin{cases}
\rho_{\text{dark}} / p_{\text{dark}}, & p_{\text{dark}} > 0, \\[6pt]
\rho_{\text{dark}}, & \text{otherwise}.
\end{cases}
$$

- $p_{\text{dark}}$: population in the dark manifold at storage time.  
- $\tilde{\rho}_{\text{dark}}$: conditional state given that the system is in the dark subspace.

### 5. Comparing to the Ideal Input Qubit

Using the ideal input qubit density matrix $\rho_{\text{in}}$ (from Cell 8), the **dark-basis fidelity** is:  
$$
F_{\text{dark}} = \operatorname{Tr}\bigl( \tilde{\rho}_{\text{dark}}\, \rho_{\text{in}} \bigr).
$$

This has a clear operational meaning:  
- $\rho_{\text{in}}$: the logical qubit we intended to store.  
- $\tilde{\rho}_{\text{dark}}$: the actual stored state **restricted to the physically correct 2D dark subspace**.  
- $F_{\text{dark}}$: how faithfully the intended qubit sits inside that dark manifold.

The cell prints:  
- `F_dark = F_{\text{dark}}`  
- `tr_dark = p_{\text{dark}}`

### 6. Why This Matters for Option C (Non-Isometric Encoding)

For **Option C**, we want to know:

1. **Is the population mostly in the dark manifold?**  
   $p_{\text{dark}} \approx 1 \quad \Rightarrow \quad$ good adiabatic transfer into the DFS.

2. **Does the dark manifold faithfully encode the qubit, or does it collapse everything onto a single direction?**  
   - If $p_{\text{dark}}$ is high **but** $F_{\text{dark}}$ is close to 1 **only for one special input** and low for others →  
     the dark manifold acts as a **projector onto one preferred dark mode** → **rank-1 channel even inside the DFS**.  
   - (Hypothetically) If we ever saw $p_{\text{dark}} \approx 1$ **and** $F_{\text{dark}} \approx 1$ for many inputs → genuine isometric SU(2) storage.

In our baseline single-control tripod:  
- $p_{\text{dark}} \approx 0.97–0.99$: excellent adiabatic storage into the dark manifold.  
- **But** fidelity and Bloch diagnostics show the reachable part of this dark manifold is **effectively 1D** from the input’s perspective.

**This cell is the core evidence that:**

> “The single-control tripod is not just ‘inefficient’ — it is **fundamentally non-isometric** as a qubit memory, **even when judged in its own correct dark-state basis**.”

That is precisely the starting point for **Option C**: treat this non-isometric, rank-deficient encoding as a **feature to be modeled** (via an induced metric) rather than a bug to be fixed.

In [ ]:
# ============================================
# CELL 9 — Tripod Dark-Manifold & Adiabaticity
# - eigen-decompose H(t) during WRITE
# - identify dark eigenstates (≈ no |e> component)
# - build dark projector on ground manifold
# - compute projector-based adiabaticity
# - compute dark-support of logical + stored directions
# ============================================
def analyze_tripod_dark_manifold(
    t_array,
    result,
    params,
    H_func,
    Omega_0_func,
    Omega_minus1_func,
    Omega_plus1_func,
    omega_rel_threshold=1e-3,
    excited_weight_tol=1e-4
):
    # Restrict to WRITE window
    T_write = params["T_write"]
    write_mask = t_array <= T_write
    t_write = t_array[write_mask]
    idx_write = np.where(write_mask)[0]

    # Precompute pulses
    Om0   = np.array([Omega_0_func(t)   for t in t_write])
    Omm1  = np.array([Omega_minus1_func(t) for t in t_write])
    Omp1  = np.array([Omega_plus1_func(t)  for t in t_write])
    Omega_tot = np.sqrt(np.abs(Om0)**2 + np.abs(Omm1)**2 + np.abs(Omp1)**2)
    Omega_max = np.max(Omega_tot) if np.max(Omega_tot) > 0 else 1.0
    Omega_thresh = omega_rel_threshold * Omega_max
    valid = Omega_tot > Omega_thresh

    # Basis indices
    idx_e   = params["idx_e"]
    idx_gm1 = params["idx_gm1"]
    idx_g0  = params["idx_g0"]
    idx_gp1 = params["idx_gp1"]

    # Store dark projectors in ground manifold
    P_D_g_list = []
    gaps = []
    num_dark_modes = []

    # Logical direction in ground manifold: (c_plus, 0, c_minus)
    c_plus  = params["c_plus"]
    c_minus = params["c_minus"]
    v_logical = np.array([c_plus, 0.0 + 0.0j, c_minus], dtype=complex)
    if np.linalg.norm(v_logical) > 0:
        v_logical /= np.linalg.norm(v_logical)

    dark_support_logical = np.zeros_like(t_write, dtype=float)
    dark_support_stored  = np.zeros_like(t_write, dtype=float)

    # Loop over times in WRITE window
    for j, (t, kglob) in enumerate(zip(t_write, idx_write)):
        Ht = H_func(t).full()
        evals, evecs = np.linalg.eigh(Ht)

        # Identify dark eigenstates: negligible |e> component
        dark_idx   = []
        bright_idx = []
        for m in range(4):
            if np.abs(evecs[idx_e, m])**2 < excited_weight_tol:
                dark_idx.append(m)
            else:
                bright_idx.append(m)
        num_dark_modes.append(len(dark_idx))

        if len(dark_idx) == 0:
            P_D_g_list.append(np.zeros((3, 3), dtype=complex))
            gaps.append(0.0)
            continue

        # Build full dark projector
        P_full = np.zeros((4, 4), dtype=complex)
        for m in dark_idx:
            v = evecs[:, m].reshape((4, 1))
            P_full += v @ v.conj().T

        # Restrict to ground manifold (rows/cols 1,2,3)
        P_g = P_full[1:, 1:]                     # 3×3

        # Clean projector numerically via eigen-decomposition
        eig_vals, eig_vecs = np.linalg.eigh(P_g)
        mask = eig_vals > 0.5
        if np.any(mask):
            V = eig_vecs[:, mask]                # columns = dark basis in ground
            P_Dg = V @ V.conj().T
        else:
            P_Dg = np.zeros((3, 3), dtype=complex)
        P_D_g_list.append(P_Dg)

        # Gap between dark and bright eigenvalues
        if bright_idx:
            dvals = np.array([evals[m] for m in dark_idx])
            bvals = np.array([evals[m] for m in bright_idx])
            gap = np.min(np.abs(dvals[:, None] - bvals[None, :]))
            gaps.append(float(np.real_if_close(gap)))
        else:
            gaps.append(0.0)

        # Dark support of logical direction
        vL = v_logical.reshape((3, 1))
        dark_support_logical[j] = np.real(vL.conj().T @ P_Dg @ vL).item()

        # Stored direction proxy from density matrix
        rho_t = result.states[kglob].full()
        vS = np.array([
            rho_t[idx_gm1, idx_g0],
            rho_t[idx_g0,  idx_g0],
            rho_t[idx_gp1, idx_g0]
        ], dtype=complex)

        if np.linalg.norm(vS) > 0:
            vS = (vS / np.linalg.norm(vS)).reshape((3, 1))
            dark_support_stored[j] = np.real(vS.conj().T @ P_Dg @ vS).item()
        else:
            dark_support_stored[j] = 0.0

    P_D_g = np.array(P_D_g_list)
    gaps = np.array(gaps)
    num_dark_modes = np.array(num_dark_modes)

    # Adiabaticity: ||dP/dt|| / gap
    adiabaticity = np.full_like(t_write, np.nan, dtype=float)
    if len(t_write) > 2:
        dt = t_write[1] - t_write[0]
        dP = []
        for i in range(len(P_D_g)):
            if i == 0 or i == len(P_D_g) - 1:
                dP.append(np.zeros((3, 3), dtype=complex))
            else:
                dP.append((P_D_g[i+1] - P_D_g[i-1]) / (2 * dt))
        dP = np.array(dP)
        dP_norm = np.array([np.linalg.norm(mat, ord="fro") for mat in dP])
        with np.errstate(divide="ignore", invalid="ignore"):
            adiabaticity = dP_norm / (np.abs(gaps) + 1e-12)

    # Bright-state composition in ground manifold, where defined
    B_weights = np.zeros((len(t_write), 3), dtype=float)
    for i, ok in enumerate(valid):
        if not ok:
            continue
        if Omega_tot[i] == 0:
            continue
        B_weights[i, 0] = np.abs(Omm1[i])**2 / Omega_tot[i]**2
        B_weights[i, 1] = np.abs(Om0[i])**2   / Omega_tot[i]**2
        B_weights[i, 2] = np.abs(Omp1[i])**2 / Omega_tot[i]**2

    # Quick plots
    fig, axes = plt.subplots(2, 2, figsize=(14, 10))

    # Bright composition
    ax = axes[0, 0]
    ax.plot(t_write, B_weights[:, 0], label='|<g-1|B>|^2')
    ax.plot(t_write, B_weights[:, 1], label='|<g0|B>|^2')
    ax.plot(t_write, B_weights[:, 2], label='|<g+1|B>|^2')
    ax.set_title("Bright-State Composition (WRITE)")
    ax.set_xlabel("Time (μs)")
    ax.set_ylabel("Weight")
    ax.set_ylim([-0.02, 1.02])
    ax.grid(True, alpha=0.3)
    ax.legend()

    # Adiabaticity
    ax = axes[0, 1]
    ax.semilogy(t_write, np.abs(adiabaticity), "r-")
    ax.axhline(0.1, color="k", linestyle="--", alpha=0.5)
    ax.set_title("Projector-Based Adiabaticity |dP/dt| / gap")
    ax.set_xlabel("Time (μs)")
    ax.set_ylabel("Adiabaticity")
    ax.grid(True, which="both", alpha=0.3)

    # Dark-subspace support
    ax = axes[1, 0]
    ax.plot(t_write, dark_support_logical, "b-",  label="Logical direction")
    ax.plot(t_write, dark_support_stored,  "g--", label="Stored proxy")
    ax.set_title("Dark-Subspace Support (WRITE)")
    ax.set_xlabel("Time (μs)")
    ax.set_ylabel("⟨v|P_D|v⟩")
    ax.set_ylim([-0.02, 1.02])
    ax.grid(True, alpha=0.3)
    ax.legend()

    # Time-dependent total Ω(t)
    ax = axes[1, 1]
    ax.plot(t_write, Omega_tot, "k-")
    ax.set_title("Total Tripod Coupling Ω_tot (WRITE)")
    ax.set_xlabel("Time (μs)")
    ax.set_ylabel("|Ω_tot(t)|")
    ax.grid(True, alpha=0.3)

    plt.tight_layout()
    plt.show()

    # Console summary
    print("\n=== Tripod Dark-Manifold Summary (WRITE) ===")
    finite = adiabaticity[np.isfinite(adiabaticity)]
    if finite.size > 0:
        print(" Mean adiabaticity =", np.nanmean(finite))
        print(" Max adiabaticity =", np.nanmax(finite))
    else:
        print(" Adiabaticity undefined (no finite gaps).")
    print(" Final logical dark-support =", dark_support_logical[-1])
    print(" Final stored dark-support =", dark_support_stored[-1])

    return {
        "t_write": t_write,
        "Omega_tot": Omega_tot,
        "B_weights": B_weights,
        "adiabaticity": adiabaticity,
        "dark_support_logical": dark_support_logical,
        "dark_support_stored": dark_support_stored,
        "num_dark_modes": num_dark_modes,
        "gaps": gaps,
    }


dark_diag = analyze_tripod_dark_manifold(
    t_array,
    result,
    params,
    H_func,
    Omega_0_func,
    Omega_minus1_func,
    Omega_plus1_func
)

🟩 **Cell 10 — Bloch-Sphere Comparison: Visualising Channel Distortion**

This cell translates the **2×2 qubit density matrices** into **Bloch vectors** and plots them on the Bloch sphere to give a **geometric picture** of how the tripod channel acts on the input qubit.

### 1. Bloch Vector Representation of a Qubit

Any single-qubit density matrix $\rho$ can be written as:
$$
\rho = \frac{1}{2} \bigl( \mathbb{I} + r_x \sigma_x + r_y \sigma_y + r_z \sigma_z \bigr),
$$
where $\sigma_x, \sigma_y, \sigma_z$ are the Pauli matrices and  
$\vec{r} = (r_x, r_y, r_z)$ is the **Bloch vector** with $\|\vec{r}\| \leq 1$.

The helper function `qubit_to_bloch(rho)` computes:
$$
r_x = \operatorname{Re} \operatorname{Tr}(\rho \sigma_x), \quad
r_y = \operatorname{Re} \operatorname{Tr}(\rho \sigma_y), \quad
r_z = \operatorname{Re} \operatorname{Tr}(\rho \sigma_z),
$$
and returns the real Bloch vector $(r_x, r_y, r_z)$.

### 2. Input vs Stored Qubit

From the WRITE-stage analysis (Cell 8), we have:

- $\rho_{\text{in}}$: the **ideal input qubit** on the logical basis $\{|g_{-1}\rangle, |g_{+1}\rangle\}$,
- $\rho_{\text{out}} = \rho_{\text{logical_norm}}$: the **normalised stored logical state** obtained by extracting and trace-normalising the 2×2 block on $\{|g_{-1}\rangle, |g_{+1}\rangle\}$.

The function `plot_bloch_sphere_comparison(analysis)`:

1. Computes the Bloch vectors:
   $$
   \vec{r}_{\text{in}} = \texttt{qubit_to_bloch}(\rho_{\text{in}}), \quad
   \vec{r}_{\text{out}} = \texttt{qubit_to_bloch}(\rho_{\text{out}}),
   $$
   and their difference:
   $$
   \Delta \vec{r} = \vec{r}_{\text{out}} - \vec{r}_{\text{in}}.
   $$

2. Plots both vectors as arrows on a unit Bloch sphere:
   - **purple**: input $\vec{r}_{\text{in}}$,
   - **orange**: stored $\vec{r}_{\text{out}}$.

This gives an immediate visual indication of:

- **Rotation**: $\vec{r}_{\text{out}}$ is a rotated version of $\vec{r}_{\text{in}}$ with similar length → approximately **unitary** channel.
- **Contraction**: $\|\vec{r}_{\text{out}}\| < \|\vec{r}_{\text{in}}\|$ → **depolarising / dephasing** in the logical subspace.
- **Collapse**: $\vec{r}_{\text{out}}$ tends to a fixed point regardless of $\vec{r}_{\text{in}}$ → effectively **rank-1** channel.

The cell prints:

```text
Input Bloch vector:   r_in
Stored Bloch vector:  r_out
Δ Bloch:              r_diff

In [ ]:
# ============================================
# CELL 10 — Bloch-Sphere Comparison
# - input 2×2 qubit vs stored 2×2 logical block
# ============================================

from mpl_toolkits.mplot3d import Axes3D # noqa: F401


def qubit_to_bloch(rho):
 """Return real Bloch vector (rx, ry, rz) from a 2×2 density matrix rho."""
 sx = np.array([[0, 1], [1, 0]], dtype=complex)
 sy = np.array([[0, -1j], [1j, 0]], dtype=complex)
 sz = np.array([[1, 0], [0, -1]], dtype=complex)
 return np.array([
 np.real(np.trace(rho @ sx)),
 np.real(np.trace(rho @ sy)),
 np.real(np.trace(rho @ sz)),
 ])


def plot_bloch_sphere_comparison(analysis):
 rho_in = analysis["rho_in_qubit"]
 rho_out = analysis["rho_logical_norm"]

 r_in = qubit_to_bloch(rho_in)
 r_out = qubit_to_bloch(rho_out)
 r_diff = r_out - r_in

 fig = plt.figure(figsize=(8, 8))
 ax = fig.add_subplot(111, projection="3d")

 # Sphere
 u = np.linspace(0, 2*np.pi, 40)
 v = np.linspace(0, np.pi, 20)
 x = np.outer(np.cos(u), np.sin(v))
 y = np.outer(np.sin(u), np.sin(v))
 z = np.outer(np.ones_like(u), np.cos(v))
 ax.plot_surface(x, y, z, color="lightgray", alpha=0.1, linewidth=0)

 # Input / output vectors
 ax.quiver(0, 0, 0, *r_in, color="purple", linewidth=3, label="Input")
 ax.quiver(0, 0, 0, *r_out, color="orange", linewidth=3, label="Stored")

 # Axes
 ax.quiver(0, 0, 0, 1, 0, 0, color="r", alpha=0.5)
 ax.quiver(0, 0, 0, 0, 1, 0, color="g", alpha=0.5)
 ax.quiver(0, 0, 0, 0, 0, 1, color="b", alpha=0.5)

 ax.set_xlim([-1, 1])
 ax.set_ylim([-1, 1])
 ax.set_zlim([-1, 1])
 ax.set_xlabel("x")
 ax.set_ylabel("y")
 ax.set_zlabel("z")
 ax.set_title("Bloch-Sphere: Input vs Stored Logical Qubit")
 ax.legend()
 plt.tight_layout()
 plt.show()

 return r_in, r_out, r_diff


r_in, r_out, r_diff = plot_bloch_sphere_comparison(analysis)
print("Input Bloch vector: ", r_in)
print("Stored Bloch vector:", r_out)
print("Δ Bloch :", r_diff)


🟩 **Cell 11 — SU(3) Gell-Mann Diagnostics on the Ground-Manifold State**

This cell decomposes the **3×3 ground-manifold density matrix** into the **SU(3) generator basis** (Gell-Mann matrices) to characterise **how the tripod uses the full qutrit** $\{|g_{-1}\rangle, |g_0\rangle, |g_{+1}\rangle\}$.

It is the bridge between:  
- the **qubit view** (only $|g_{-1}\rangle, |g_{+1}\rangle$), and  
- the **qutrit view** (full SU(3) structure of the tripod manifold).

### 1. Ground-Manifold Density Matrix

From the WRITE-stage analysis (Cell 8), we extracted:  
$$
\rho_3 \equiv \rho_{\text{ground_3x3}} =
\begin{pmatrix}
\rho_{-1,-1} & \rho_{-1,0}   & \rho_{-1,+1} \\
\rho_{0,-1}   & \rho_{0,0}     & \rho_{0,+1} \\
\rho_{+1,-1}  & \rho_{+1,0}   & \rho_{+1,+1}
\end{pmatrix},
$$
in the ordered basis $\{|g_{-1}\rangle,\ |g_0\rangle,\ |g_{+1}\rangle\}$.

This $\rho_3$ contains **all population and coherence information** in the ground manifold after WRITE, including leakage into $|g_0\rangle$.

### 2. SU(3) Expansion: Gell-Mann Basis

Any Hermitian traceless 3×3 matrix (and thus any qutrit density matrix) can be expanded as:  
$$
\rho_3 = \frac{1}{3}\,\mathbb{I}_3 + \frac{1}{2} \sum_{i=1}^{8} r_i \, \lambda_i,
$$
where:
- $\mathbb{I}_3$ is the 3×3 identity,
- $\{\lambda_i\}_{i=1}^8$ are the **Gell-Mann matrices** (the eight SU(3) generators),
- $\vec{r} = (r_1,\dots,r_8)$ is the **generalised Bloch vector** in SU(3) space.

This cell computes the real coefficients:  
$$
\langle \lambda_i \rangle = \operatorname{Re} \operatorname{Tr}(\rho_3 \, \lambda_i), \quad i = 1,\dots,8,
$$
stores them as `su3_comps`, and plots them as a bar chart — giving a **fingerprint** of the ground-manifold state in SU(3) space.

### 3. Physical Interpretation of the Components

Each Gell-Mann matrix probes a specific pattern:

| $\lambda_i$ | Probes |
|-------------|--------|
| $\lambda_1, \lambda_2$ | coherence & population imbalance between $|g_{-1}\rangle$ and $|g_0\rangle$ |
| $\lambda_4, \lambda_5$ | **logical qubit coherence** between $|g_{-1}\rangle$ and $|g_{+1}\rangle$ |
| $\lambda_6, \lambda_7$ | coherence between $|g_0\rangle$ and $|g_{+1}\rangle$ |
| $\lambda_3, \lambda_8$ | diagonal population imbalances (generalised “Z” for SU(3)) |

**Qualitatively**:
- Large $\langle\lambda_4\rangle, \langle\lambda_5\rangle$: strong **qubit-like** coherence.
- Large values in $\lambda_1, \lambda_2, \lambda_6, \lambda_7$: **genuine qutrit** character / leakage involving $|g_0\rangle$.
- Dominant $\lambda_3, \lambda_8$: population heavily skewed → **effective rank-1** behaviour.

### 4. (Non-Isometric Encoding)

- The 8-component vector $\vec{r}$ tells you **which directions in qutrit space** the memory actually populates.
- Comparing $\vec{r}$ across different input qubits reveals **how the tripod warps** the logical Bloch sphere into the full SU(3) manifold.
- If many different inputs produce **nearly the same SU(3) fingerprint**, that is another clear signature of an **effective rank-1 channel** — the memory collapses the entire Bloch sphere into a narrow ray in SU(3) space.

When you later implement **SU(3) / holonomic gates** on the ground manifold, this diagnostic becomes the **natural language** for:

- defining an **induced metric $G$** on the logical subspace via its embedding into SU(3),
- designing gates that act correctly on the **distorted Bloch ball** rather than assuming an ideal qubit.

In short, **Cell 11** converts the abstract “the tripod lives in 3D” into a **concrete, eight-component fingerprint** you can track, compare, and eventually use to build metric-aware quantum control.

In [ ]:
# ============================================
# CELL 11 — SU(3) Gell-Mann Diagnostics (3×3)
# ============================================

def get_gellmann_matrices():
 λ1 = np.array([[0, 1, 0], [1, 0, 0], [0, 0, 0]], dtype=complex)
 λ2 = np.array([[0, -1j, 0], [1j, 0, 0], [0, 0, 0]], dtype=complex)
 λ3 = np.array([[1, 0, 0], [0, -1, 0], [0, 0, 0]], dtype=complex)
 λ4 = np.array([[0, 0, 1], [0, 0, 0], [1, 0, 0]], dtype=complex)
 λ5 = np.array([[0, 0, -1j], [0, 0, 0], [1j, 0, 0]], dtype=complex)
 λ6 = np.array([[0, 0, 0], [0, 0, 1], [0, 1, 0]], dtype=complex)
 λ7 = np.array([[0, 0, 0], [0, 0, -1j], [0, 1j, 0]], dtype=complex)
 λ8 = (1/np.sqrt(3)) * np.array([[1, 0, 0], [0, 1, 0], [0, 0, -2]], dtype=complex)
 return [λ1, λ2, λ3, λ4, λ5, λ6, λ7, λ8]


def plot_su3_ground_block(analysis):
 rho3 = analysis["rho_ground_3x3"]
 G = get_gellmann_matrices()
 comps = np.array([np.real(np.trace(rho3 @ L)) for L in G])

 plt.figure(figsize=(8, 4))
 plt.bar(np.arange(1, 9), comps)
 plt.xlabel("Gell-Mann index")
 plt.ylabel("⟨λᵢ⟩")
 plt.title("SU(3) Ground-Manifold Expectation Values")
 plt.grid(axis="y", alpha=0.3)
 plt.tight_layout()
 plt.show()

 return comps


su3_comps = plot_su3_ground_block(analysis)
print("SU(3) components:", su3_comps)


🟩 **Cell 12 — Final WRITE-Stage Summary (Metrics + Dark-Manifold Diagnostics)**

This cell gathers **all diagnostics** from previous steps into a **single, human-readable summary** of what the WRITE stage has actually done to the input qubit.

It combines:

- Qubit-level metrics (Bloch vectors, fidelity, efficiency),
- Dark-manifold & adiabaticity diagnostics,
- Full qutrit-level SU(3) fingerprint.

This is the **verdict cell** for the baseline single-tripod memory.

### 1. Bloch Vector Comparison: Input vs Stored Qubit

From Cell 10:

- Input Bloch vector: $\mathbf{r}_{\text{in}} \in \mathbb{R}^3$
- Stored Bloch vector: $\mathbf{r}_{\text{out}} \in \mathbb{R}^3$
- Difference: $\Delta \mathbf{r} = \mathbf{r}_{\text{out}} - \mathbf{r}_{\text{in}}$

**Interpretation**:
- $\|\Delta \mathbf{r}\| \approx 0$: memory acts nearly as **identity** on this input.
- Large $\|\Delta \mathbf{r}\|$: strong **distortion** → clear evidence of **non-isometry**.

In **Option C**, collecting $\Delta \mathbf{r}$ over many inputs reveals **how anisotropic and input-dependent** the channel really is.

### 2. Storage Fidelity and Efficiency

From Cell 8:

- **Storage fidelity** (conditional, on logical subspace):  
  $$
  F_{\text{store}} = \operatorname{Tr}(\rho_{\text{out}} \, \rho_{\text{in}})
  $$

- **Storage efficiency** (population in logical subspace):  
  $$
  \eta_{\text{store}} = \operatorname{Tr}(\rho_{\text{logical_2×2}}) \quad \in [0,1]
  $$

**Classic rank-1 signature**:  
High $\eta_{\text{store}}$ **but** low $F_{\text{store}}$ →  
> “We stored *something* efficiently, but **not the right qubit**.”

### 3. Dark-Manifold Adiabaticity

From Cell 9, the instantaneous adiabaticity parameter:  
$$
\mathcal{A}(t) \equiv \frac{\| \dot{P}_D(t) \|_{\text{Fro}}}{\Delta_{\text{gap}}(t)},
$$
where $P_D(t)$ is the dark projector and $\Delta_{\text{gap}}(t)$ is the min gap to bright states.

Summary prints over the WRITE window:
- Mean adiabaticity: $\langle \mathcal{A} \rangle$
- Worst-case: $\max \mathcal{A}$

**Interpretation**:
- $\mathcal{A} \ll 1$ everywhere → **adiabatic theorem satisfied**.
- If rank-1 collapse still occurs under $\mathcal{A} \ll 1$, the limitation is **structural** (geometry/CGCs), **not** non-adiabatic transitions.

This is crucial for **Option C**: you prove the non-isometry is **intrinsic**, not a control error.

### 4. Dark-Subspace Support at End of WRITE

From Cell 9:

- Logical direction support:  
  $\langle v_{\text{logical}} | P_D(T_{\text{write}}) | v_{\text{logical}} \rangle$
- Stored proxy support (from actual $\rho$):  
  $\langle v_{\text{stored}} | P_D(T_{\text{write}}) | v_{\text{stored}} \rangle$

Both printed as values in $[0,1]$.

**Interpretation**:
- $\approx 1$: the vector lies almost entirely in the dark manifold.
- Logical support $\ll 1$ while stored support $\approx 1$ → the system **projected onto a different dark eigenvector** than the intended logical one → another smoking gun for rank-1 behaviour.

### 5. SU(3) Components (Ground-Manifold Fingerprint)

Printed as eight real numbers:  
$$
\langle \lambda_i \rangle = \operatorname{Re} \operatorname{Tr}(\rho_3 \lambda_i), \quad i = 1,\dots,8
$$

These tell you:
- Whether the state is **aligned along one dominant SU(3) direction** (rank-1 like),
- Or genuinely spread across the qutrit (more isotropic).

These components become the **natural coordinates** for any future SU(3)/holonomic gates in Option C.

### 6. Why This Cell Matters for the Overall Narrative

For your **baseline single-tripod simulation**, this cell lets you state clearly:

- “The process is adiabatic.”
- “Population is efficiently transferred to the dark manifold.”
- “Yet the logical qubit is heavily distorted / projected.”
- “The ground manifold behaves effectively as rank-1 in SU(3) space.”

When you later run the **parameter-sweep cell** (varying CGCs, phases, detunings, pulse shapes), these **same summary numbers** are what you monitor to prove:

> “**No realistic single-control tripod configuration** restores a full-rank, approximately isometric qubit memory.  
> The non-isometry is **structural**, not a tuning artefact.”

**Cell 12 is therefore the diagnostic anchor** of your entire “intrinsic rank limitation” argument.

In [ ]:
# ============================================
# CELL 12 — Final WRITE-Stage Summary
# ============================================
def write_stage_summary(analysis, dark_diag, r_in, r_out, r_diff, su3_comps):
    print("\n================ WRITE-STAGE SUMMARY ================")
    print("\nBloch vectors:")
    print(" Input r  =", np.round(r_in, 4))
    print(" Stored r =", np.round(r_out, 4))
    print(" Δr       =", np.round(r_diff, 4))

    print("\nStorage metrics:")
    print(f" Storage fidelity F = {analysis['storage_fidelity']:.6f}")
    print(f" Storage efficiency η = {analysis['storage_efficiency']:.6f}")

    finite = dark_diag["adiabaticity"][np.isfinite(dark_diag["adiabaticity"])]
    if finite.size > 0:
        print("\nDark-manifold adiabaticity:")
        print(f" Mean |dP/dt|/gap = {np.nanmean(finite):.3e}")
        print(f" Max  |dP/dt|/gap = {np.nanmax(finite):.3e}")
    else:
        print("\nDark-manifold adiabaticity: undefined (no finite gaps).")

    print("\nDark-subspace support at end of WRITE:")
    print(f" Logical direction: {dark_diag['dark_support_logical'][-1]:.4f}")
    print(f" Stored proxy     : {dark_diag['dark_support_stored'][-1]:.4f}")

    print("\nSU(3) ground-manifold components (⟨λᵢ⟩):")
    for i, val in enumerate(su3_comps, start=1):
        print(f" λ{i}: {val:.6f}")

    print("=====================================================\n")


write_stage_summary(analysis, dark_diag, r_in, r_out, r_diff, su3_comps)

🟩 **Cell 13 — Instantaneous Dark-Space Inspection at \( t = T_{\text{write}} \)**

This diagnostic cell performs a **local, eigenbasis-level check** of the tripod Hamiltonian at the **storage time** \( t = T_{\text{write}} \). It is deliberately simple and “low-level” compared to the projector-based dark-manifold analysis.

### 1. What the Cell Actually Computes

At the time \( t_{\text{store}} = T_{\text{write}} \), it:

1. Builds the instantaneous Hamiltonian:  
   $$
   H(t_{\text{store}}) = H_s,
   $$
   using the same time-dependent function `H_func(t)` as the master-equation solver.

2. Diagonalises it:  
   $$
   H_s \, |E_m\rangle = E_m \, |E_m\rangle, \quad m = 0,1,2,3,
   $$
   yielding eigenvalues $\{E_m\}$ and eigenvectors $|E_m\rangle$.

3. For each eigenvector, prints the **excited-state component**:  
   $$
   |\langle e | E_m \rangle|^2.
   $$

**Interpretation**:
- $|\langle e | E_m \rangle|^2 \approx 0$ → that eigenstate is effectively **dark**.
- $|\langle e | E_m \rangle|^2 \gg 0$ → that eigenstate is **bright**.

This gives you a **direct, brute-force view** of how many dark eigenmodes exist at storage time and how clean they are numerically.

### 2. Why This Is Useful Alongside the Projector-Based Diagnostics

Cell 9 constructed the **dark projector** $P_D(t)$ by:
- identifying dark eigenstates via tiny $|e\rangle$-component,
- restricting to the ground manifold,
- using it to compute dark support, adiabaticity, etc.

**Cell 13** complements that with a **raw snapshot**:
- You immediately see **how many** eigenstates are truly dark at $t = T_{\text{write}}$.
- You see **how small** the residual $|e\rangle$-leakage really is (numerical confirmation of the dark subspace quality).
- You can verify whether there are 0, 1, or 2 “good” dark eigenstates.

In the **single-tripod baseline**, you typically observe:
- **One** eigenvector with nearly all weight in $|e\rangle$ → bright state,
- **Two** eigenvectors with $|\langle e | E_m \rangle|^2 \lesssim 10^{-10}$ → mathematically dark,
- But (as shown by previous diagnostics) **only one** of these dark eigenstates is dynamically populated from $|g_0\rangle$ under the given pulses.

This is fully consistent with the **effective rank-1 channel** diagnosed elsewhere.

### 3. How This Supports the “No Hidden Second Dark Mode” Claim

When you later run the **parameter-sweep cell** (varying CGCs, relative phases, detunings, pulse shapes, etc.), you can:

- Re-run this instantaneous eigen-analysis for every parameter point,
- Confirm that **no realistic parameter choice** suddenly creates a **clean, dynamically reachable 2D dark subspace** that would support an isometric SU(2) memory.

If, across the entire sweep:
- The instantaneous eigen-spectrum always shows **two mathematically dark states**,
- But **only one** is ever significantly populated (as proven by fidelity, Bloch, SU(3), and dark-support diagnostics),

then you have ironclad evidence that:

> The effective memory channel is **structurally rank-1** for a single-control tripod under realistic constraints — **not** merely a result of poor tuning.

**Cell 13** is therefore a **critical sanity-check and forensic tool** for the dark-space structure exactly at the moment of storage.

In [ ]:
# =====================================
# DARK SPACE DIAGNOSTIC
# =====================================
def inspect_dark_space(H_func, params, t_array):
    t_store = params["T_write"]
    Hs = H_func(t_store).full()
    evals, evecs = np.linalg.eigh(Hs)
    idx_e = params["idx_e"]

    print("Eigenvalues:")
    print(evals)

    print("\n|e>-amplitudes of each eigenvector:")
    for m in range(4):
        print(m, abs(evecs[idx_e, m])**2)

    print("\nInterpretation:")
    print("Small |e>-component → dark; large → bright.")


inspect_dark_space(H_func, params, t_array)

In [ ]:
plt.plot(t_array, np.abs([Omega_minus1_func(t) for t in t_array]), label="|Ω_-1|")
plt.plot(t_array, np.abs([Omega_plus1_func(t) for t in t_array]), label="|Ω_+1|")
plt.plot(t_array, np.abs([Omega_0_func(t) for t in t_array]), label="|Ω_0|")
plt.legend()
plt.grid()
plt.show()


🟩 **Cell 15 — Final Dark-Space / Adiabaticity Sanity Check**

This cell prints **three key scalar diagnostics** evaluated at the **end of the WRITE stage**:

$$
t_{\text{store}} = T_{\text{write}}.
$$

### 1. Final Logical Dark Support

$$
\text{dark\_support\_logical}[-1]
= 
\langle v_{\text{logical}} | P_D(t_{\text{store}}) | v_{\text{logical}} \rangle
$$

where:
- $|v_{\text{logical}}\rangle \propto (c_{+}, 0, c_{-})^{\mathsf{T}}$ is the **ideal logical qubit direction** embedded in the ground manifold $\{|g_{-1}\rangle, |g_0\rangle, |g_{+1}\rangle\}$,
- $P_D(t)$ is the **instantaneous dark projector** in the ground subspace (constructed in `analyze_tripod_dark_manifold`).

**Interpretation**:
- $\approx 1$ → the intended logical qubit lies almost entirely in the dark subspace (good!).
- $\ll 1$ → part of the logical qubit is bright → lossy or non-adiabatic.

### 2. Final Stored-State Dark Support

$$
\text{dark\_support\_stored}[-1]
=
\langle v_{\text{stored}} | P_D(t_{\text{store}}) | v_{\text{stored}} \rangle
$$

where $|v_{\text{stored}}\rangle$ is a **proxy direction** extracted directly from the stored density matrix, typically built from coherences with $|g_0\rangle$:

$$
|v_{\text{stored}}\rangle 
\propto 
\begin{pmatrix}
\rho_{g_{-1}, g_0}(t_{\text{store}}) \\
\rho_{g_0, g_0}(t_{\text{store}}) \\
\rho_{g_{+1}, g_0}(t_{\text{store}})
\end{pmatrix}.
$$

**Interpretation**:
- Quantifies **where the system actually placed the spin-wave** relative to the true dark manifold.
- If **both** logical and stored supports are high and close to each other → successful dark-state storage of the intended mode.
- In the baseline single-tripod, you typically see **high stored support** but **poor logical support**, confirming projection onto a **different dark eigenmode** than intended.

### 3. Maximum Adiabaticity Parameter Over WRITE Window

$$
\max_t \left| \frac{dP_D(t)}{dt} \right| \Big/ \operatorname{gap}(t)
=
\max_t \, \text{adiabaticity}(t)
$$

where the adiabaticity parameter is:

$$
\text{adiabaticity}(t) = 
\frac{\| dP_D(t)/dt \|_{\mathrm{F}}}{\operatorname{gap}(t) + \varepsilon},
$$

with:
- $\| \cdot \|_{\mathrm{F}}$: Frobenius norm,
- $\operatorname{gap}(t)$: minimum instantaneous gap between dark and bright eigenvalues,
- $\varepsilon$: small regularisation.

**Interpretation**:
- $\max_t \, \text{adiabaticity}(t) \ll 1$ → evolution is **adiabatic** w.r.t. dark–bright splitting.
- Crucially: if adiabaticity is good **but rank-1 collapse still occurs**, the limitation is **structural** (shared control leg, CGC geometry), **not** non-adiabatic leakage.

### 4. Why This Minimal Cell Matters

These **three numbers** act as a final **sanity check** that distils the entire dark-space analysis:

| Question                                      | Diagnostic                          | Good value |
|--------------------------------------------- |-------------------------------------|------------|
| Is the logical qubit direction dark?         | logical dark support                | ≈ 1        |
| Is the actually stored excitation dark?      | stored dark support                 | ≈ 1        |
| Was the dark manifold followed adiabatically?| max adiabaticity                    | ≪ 1        |

In the **baseline single-tripod**, you typically see:

- high stored dark support,
- high adiabaticity,
- **but** poor logical dark support or severe distortion in fidelity/Bloch space.

→ Confirms that even under **ideal adiabatic, dark-state conditions**, the accessible dark subspace is **effectively 1D** — the memory channel is **structurally rank-1**.

This is the concise quantitative foundation for the final parameter-sweep cell that will prove **no realistic tuning restores full-rank storage**.

In [ ]:
print("Final logical dark support =", dark_diag["dark_support_logical"][-1])
print("Final stored dark support =", dark_diag["dark_support_stored"][-1])
print("Max adiabaticity =", np.nanmax(dark_diag["adiabaticity"]))


# 🟩 Final Cell — Parameter Sweep to Test for Rank Restoration

This (new) cell performs a **structured parameter sweep** over the single-tripod memory and checks whether **any reasonable choice of physical parameters** appears to restore a genuinely **rank-2 qubit channel** in the logical subspace \(\{|g_{-1}\rangle, |g_{+1}\rangle\}\).

---

## 1. Goal: Look for evidence of a true 2D logical channel

We probe the mapping:

\[
\mathcal{E}:\ \rho_{\text{in}} \in \mathcal{B}(\mathbb{C}^2_{\{\sigma^+,\sigma^-\}}) 
\;\longrightarrow\;
\rho_{\text{out}} \in \mathcal{B}(\text{span}\{|g_{-1}\rangle, |g_{+1}\rangle\}),
\]

by **feeding in multiple logical input states** and examining how their encoded outputs populate the logical subspace.  

Concretely, we use the standard polarization basis set:

- \(|\sigma^+\rangle\),
- \(|\sigma^-\rangle\),
- \(|D\rangle = (\;|\sigma^+\rangle + |\sigma^-\rangle\;)/\sqrt{2}\),
- \(|A\rangle = (\;|\sigma^+\rangle - |\sigma^-\rangle\;)/\sqrt{2}\),
- \(|R\rangle = (\;|\sigma^+\rangle + i|\sigma^-\rangle\;)/\sqrt{2}\),
- \(|L\rangle = (\;|\sigma^+\rangle - i|\sigma^-\rangle\;)/\sqrt{2}\).

For each **parameter set** and each **input state**, we:

1. Set \((c_{+}, c_{-})\) accordingly.
2. Rebuild the time-dependent pulses \(\Omega_0(t), \Omega_{-1}(t), \Omega_{+1}(t)\).
3. Run the full **master equation** (\verb|mesolve|) over the write–gate–read window.
4. Perform the WRITE-stage analysis to extract the **2×2 logical block** \(\rho^{(i)}_{\text{out}}\) on \(\{|g_{-1}\rangle, |g_{+1}\rangle\}\) at the end of the WRITE stage, normalised to unit trace.

Then we form the **average encoded state**:

\[
\rho^\star = \frac{1}{N} \sum_{i=1}^N \rho^{(i)}_{\text{out}},
\]

and compute its eigenvalues \(\lambda_0 \leq \lambda_1\).  

- If the channel behaves like an **effective rank-1 projector** (up to small noise), we expect:
  \[
  \lambda_1 \approx 1,\qquad \lambda_0 \approx 0,
  \]
  and each \(\rho^{(i)}_{\text{out}}\) will be nearly identical (high fidelity to \(\rho^\star\)).
- A genuinely **rank-2** (isometric) channel would exhibit:
  - a **spread** of outputs across the Bloch sphere, and
  - \(\rho^\star\) significantly **mixed**, with \(\lambda_0, \lambda_1\) both away from \(\{0,1\}\).

We summarise each parameter set by:

- \(\lambda_0, \lambda_1\) of \(\rho^\star\),
- \(\mathrm{Tr}[(\rho^\star)^2]\) (purity),
- \(\min_i F(\rho^{(i)}_{\text{out}}, \rho^\star)\),
- \(\max_i F(\rho^{(i)}_{\text{out}}, \rho^\star)\),

where fidelity here is simply:

\[
F(\rho, \sigma) = \mathrm{Tr}(\rho \sigma)
\]

for 2×2 states that are already very close to pure.

---

## 2. Parameters swept

To test whether **any straightforward knob** could restore a rank-2 channel, we sweep over:

1. **Clebsch–Gordan asymmetry** between \(|g_{-1}\rangle\) and \(|g_{+1}\rangle\):
   \[
   \texttt{CG\_minus1},\ \texttt{CG\_plus1} \in \{0.2, 0.4, 0.577, 0.8, 1.0\},
   \]
   with \(\texttt{CG\_0}\) fixed.

2. **Relative probe phases**:
   \[
   \phi_{+}, \phi_{-} \in \{0, \pi/2\},
   \]
   which change the relative phase of the writing of \(\sigma^\pm\) into the tripod.

3. **Optional Zeeman detunings** (you can uncomment / extend inside the cell), e.g.:
   \[
   \delta_{-1}, \delta_0, \delta_{+1} \approx \omega_L \cdot m_F,
   \]
   to test the effect of a modest \(B\)-field.

You can extend this further (e.g. scaling \(\Omega_C\), \(\Omega_P\)) if desired, but even this grid is already sufficient to see whether **any “reasonable” parameter tweak** breaks the effective rank-1 behaviour.

---

## 3. Interpretation of the printed summary

For each parameter tuple \((\texttt{CG\_minus1}, \texttt{CG\_plus1}, \phi_{+}, \phi_{-})\) the cell prints:

- \(\lambda_0, \lambda_1\) of \(\rho^\star\),
- purity \(\mathrm{Tr}[(\rho^\star)^2]\),
- \(F_{\min}, F_{\max}\) over the six input states.

**Rank-1 signature** (what we in fact observe in the baseline single-tripod):

- \(\lambda_1 \approx 1,\ \lambda_0 \ll 1\),
- purity close to 1,
- all fidelities \(F(\rho^{(i)}_{\text{out}}, \rho^\star)\) close to 1 (i.e. all outputs collapsed onto essentially a single dark eigenmode).

If, for all swept parameter combinations, the diagnostics remain in this regime, that is **strong numerical evidence** that the **rank-1 collapse is structural**, not tunable away by simple choices of CGC asymmetry, relative probe phase, or mild Zeeman shifts — exactly supporting the narrative that a **single-control tripod cannot realise a rank-2 SU(2) memory**.

You can regard this as a “no free lunch” theorem in numerical form for this geometry.

---


In [ ]:
# ============================================
# FINAL CELL — PARAMETER SWEEP FOR RANK TEST
# ============================================

from copy import deepcopy

def logical_inputs_sigma_basis():
    """
    Return a dict of logical input amplitudes (c_plus, c_minus)
    for standard polarization states in the {σ+, σ-} basis.
    """
    # σ+ = (1, 0)
    c_sp = 1.0 + 0j
    c_sm = 0.0 + 0j

    inputs = {}

    # |σ+>
    inputs["sigma_plus"]  = (1.0 + 0j, 0.0 + 0j)
    # |σ->
    inputs["sigma_minus"] = (0.0 + 0j, 1.0 + 0j)

    # |D> = (σ+ + σ-)/√2
    inputs["D"] = ((c_sp + c_sm) / np.sqrt(2),
                   (c_sp + c_sm) / np.sqrt(2))

    # |A> = (σ+ - σ-)/√2
    inputs["A"] = ((c_sp - c_sm) / np.sqrt(2),
                   (c_sp - c_sm) / np.sqrt(2))

    # |R> = (σ+ + i σ-)/√2
    inputs["R"] = ((c_sp + 1j*c_sm) / np.sqrt(2),
                   (c_sp + 1j*c_sm) / np.sqrt(2))

    # |L> = (σ+ - i σ-)/√2
    inputs["L"] = ((c_sp - 1j*c_sm) / np.sqrt(2),
                   (c_sp - 1j*c_sm) / np.sqrt(2))

    # NOTE: Above definitions effectively keep the same phase in both
    # components for D/A/R/L; for our purposes (testing rank) only the
    # relative σ+/σ- structure matters, not global phases.

    # Fix norms exactly to 1
    for key, (cp, cm) in inputs.items():
        n2 = np.abs(cp)**2 + np.abs(cm)**2
        if n2 > 0:
            s = np.sqrt(n2)
            inputs[key] = (cp/s, cm/s)
    return inputs


def run_single_config(params_base,
                      CG_minus1,
                      CG_plus1,
                      phi_plus,
                      phi_minus,
                      verbose=False):
    """
    For a single parameter configuration:
      - set CGCs and phases,
      - loop over 6 logical input states,
      - for each, run mesolve + write-stage analysis,
      - collect the normalised 2×2 logical output states,
      - build ρ★ and compute its eigenvalues, purity, and Fmin/Fmax.
    """
    # store outputs per input
    logical_inputs = logical_inputs_sigma_basis()
    rho_out_list = []

    # loop over inputs
    for label, (c_plus, c_minus) in logical_inputs.items():
        p = define_parameters()            # fresh param set
        p.update(params_base)             # inherit baseline overrides
        p["CG_minus1"] = CG_minus1
        p["CG_plus1"]  = CG_plus1
        # keep CG_0 from base (or 1/√3)
        p["c_plus"]    = c_plus
        p["c_minus"]   = c_minus
        p["phi_plus"]  = phi_plus
        p["phi_minus"] = phi_minus

        # pulses
        Omega_0, Omega_m1, Omega_p1 = define_pulse_functions(p)
        Hf      = build_hamiltonian_func(p, Omega_0, Omega_m1, Omega_p1)
        cops    = define_collapse_operators(p)

        # time grid
        t_arr = np.linspace(0.0, p["T_total"], p["N_steps"])

        # initial state (always |g0>)
        psi0 = basis(4, p["idx_g0"])
        rho0 = ket2dm(psi0)

        # evolve
        res = mesolve(Hf, rho0, t_arr, cops, [])

        # write-stage analysis
        ana = analyze_write_stage(res, t_arr, p)
        rho2n = ana["rho_logical_norm"]
        rho_out_list.append((label, rho2n))

    # build ρ★ = average over inputs
    rho_star = np.zeros((2, 2), dtype=complex)
    for _, rho2n in rho_out_list:
        rho_star += rho2n
    rho_star /= len(rho_out_list)

    # eigenvalues and purity
    evals, _ = np.linalg.eigh(rho_star)
    evals = np.sort(evals)  # ascending
    purity = np.real(np.trace(rho_star @ rho_star))

    # fidelity of each output to ρ★
    F_list = []
    for label, rho2n in rho_out_list:
        F = np.real(np.trace(rho2n @ rho_star))
        F_list.append((label, F))

    F_vals = [F for (_, F) in F_list]
    Fmin = float(np.min(F_vals))
    Fmax = float(np.max(F_vals))

    if verbose:
        print("Outputs vs ρ★ for this config:")
        for label, F in F_list:
            print(f"  {label:10s}: F = {F:.6f}")

    return {
        "evals_star": evals,
        "purity_star": purity,
        "F_list": F_list,
        "Fmin": Fmin,
        "Fmax": Fmax,
        "rho_star": rho_star,
    }


# -----------------------
# Parameter grids
# -----------------------

CG_values   = [0.2, 0.4, 0.577, 0.8, 1.0]
phi_values  = [0.0, 0.5*np.pi]   # 0 and π/2

# You can customise baseline here if desired:
params_base = define_parameters()
# Example: uncomment to add a small Zeeman splitting
# omega_L = 2*np.pi*1.0   # rad/µs
# params_base["delta_m1"] = -omega_L
# params_base["delta_0"]  = 0.0
# params_base["delta_p1"] = +omega_L

print("===========================================================")
print("  PARAMETER SWEEP: CGCs, phases, (optional) Zeeman shifts  ")
print("===========================================================\n")

sweep_results = []

for CGm in CG_values:
    for CGp in CG_values:
        for phi_p in phi_values:
            for phi_m in phi_values:
                print(f"Testing CG_minus1 = {CGm:.3f}, CG_plus1 = {CGp:.3f}, "
                      f"phi_plus = {phi_p:.2f}, phi_minus = {phi_m:.2f}")
                res_cfg = run_single_config(params_base,
                                            CGm, CGp,
                                            phi_p, phi_m,
                                            verbose=False)
                evals = res_cfg["evals_star"]
                purity = res_cfg["purity_star"]
                Fmin   = res_cfg["Fmin"]
                Fmax   = res_cfg["Fmax"]

                print(f"   eigenvalues(ρ★) = [{evals[0]:.6f}, {evals[1]:.6f}]")
                print(f"   purity Tr(ρ★²)  = {purity:.6f}")
                print(f"   Fmin = {Fmin:.6f}, Fmax = {Fmax:.6f}")
                print("-----------------------------------------------------------")

                sweep_results.append({
                    "CGm": CGm,
                    "CGp": CGp,
                    "phi_plus": phi_p,
                    "phi_minus": phi_m,
                    "evals": evals,
                    "purity": purity,
                    "Fmin": Fmin,
                    "Fmax": Fmax,
                })

print("\n===========================================================")
print("      SWEEP COMPLETE — inspect sweep_results for data       ")
print("===========================================================\n")
